In [7]:
from langchain_community.llms import Ollama
from typing import TypedDict
from pydantic import BaseModel, Field

In [8]:
llm=Ollama(model="llama3")

C:\Users\Ashish\AppData\Local\Temp\ipykernel_25720\362231852.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama3")


In [9]:
#define schema
class evalutionschema(BaseModel):
    feedback:str =Field(description="Detailed feedback for the essay.")
    score:int =Field(description='score out of 10.',ge=0,le=10)
 
 

In [10]:
structured_model=llm.with_structured_output(evalutionschema)# getting error because 'with_structured_output' for open_ai

NotImplementedError: 

# structure output with ollama

In [11]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel,Field


In [12]:
#parser
parser=PydanticOutputParser(pydantic_object=evalutionschema)

In [13]:
#promt
promt=PromptTemplate(
    template=(
        "Grade the following essay out of 10 and provide feedback.\n"
        "Essay: {essay}\n\n"
        "{format_instructions}"
    ),
    input_variables=["essay"],   # only "essay" is required
    partial_variables={          # format_instructions is auto-filled
        "format_instructions": parser.get_format_instructions()
    },
)

In [14]:
llm_model=ChatOllama(model="llama3")

# essay on today's technology

In [15]:
essay='''Technology today has become an inseparable part of our daily lives, shaping how we work, learn, communicate, and entertain ourselves. From artificial intelligence and cloud computing to biotechnology and renewable energy, innovations are transforming industries and society at an unprecedented pace.

One of the most influential advancements is artificial intelligence (AI), which powers everything from voice assistants and recommendation systems to autonomous vehicles and medical diagnostics. AI is not only improving efficiency but also enabling solutions to complex global challenges, such as climate change and healthcare accessibility.

Similarly, the rise of 5G and advanced connectivity has accelerated the digital revolution by making information and services accessible in real time, enhancing opportunities for remote work, online education, and telemedicine. Coupled with cloud computing, businesses can now scale faster, innovate more easily, and operate globally with fewer limitations.

However, while technology provides remarkable benefits, it also poses significant challenges. Issues such as data privacy, cybersecurity threats, digital addiction, and the widening digital divide must be carefully managed. Furthermore, the rapid pace of automation raises questions about the future of jobs and the need for continuous reskilling.

In conclusion, today’s technology is a double-edged sword—capable of bringing about tremendous progress but also requiring responsible use and thoughtful governance. If balanced well, it has the power to create a future that is smarter, more sustainable, and inclusive for all.
'''

#chain

In [16]:
from langchain.chains import LLMChain
chain=LLMChain(prompt=promt,llm=llm_model,output_parser=parser)

C:\Users\Ashish\AppData\Local\Temp\ipykernel_25720\2961266291.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(prompt=promt,llm=llm_model,output_parser=parser)


In [17]:
result=chain.invoke({"essay":essay})

In [18]:
print(result)

{'essay': 'Technology today has become an inseparable part of our daily lives, shaping how we work, learn, communicate, and entertain ourselves. From artificial intelligence and cloud computing to biotechnology and renewable energy, innovations are transforming industries and society at an unprecedented pace.\n\nOne of the most influential advancements is artificial intelligence (AI), which powers everything from voice assistants and recommendation systems to autonomous vehicles and medical diagnostics. AI is not only improving efficiency but also enabling solutions to complex global challenges, such as climate change and healthcare accessibility.\n\nSimilarly, the rise of 5G and advanced connectivity has accelerated the digital revolution by making information and services accessible in real time, enhancing opportunities for remote work, online education, and telemedicine. Coupled with cloud computing, businesses can now scale faster, innovate more easily, and operate globally with fe

In [20]:
result['essay']

'Technology today has become an inseparable part of our daily lives, shaping how we work, learn, communicate, and entertain ourselves. From artificial intelligence and cloud computing to biotechnology and renewable energy, innovations are transforming industries and society at an unprecedented pace.\n\nOne of the most influential advancements is artificial intelligence (AI), which powers everything from voice assistants and recommendation systems to autonomous vehicles and medical diagnostics. AI is not only improving efficiency but also enabling solutions to complex global challenges, such as climate change and healthcare accessibility.\n\nSimilarly, the rise of 5G and advanced connectivity has accelerated the digital revolution by making information and services accessible in real time, enhancing opportunities for remote work, online education, and telemedicine. Coupled with cloud computing, businesses can now scale faster, innovate more easily, and operate globally with fewer limita

In [25]:
feedback_obj = result["text"]   # This is your evalutionschema object
print("feedback:- ",feedback_obj.feedback)    # Detailed feedback
print("score:- ",feedback_obj.score)       # Score out of 10

feedback:-  The essay provides a good overview of how technology has impacted daily life, and touches on several relevant points such as AI, cloud computing, and connectivity. However, it could be improved by providing more specific examples and analysis to support the claims made.

One strength is that the essay does a good job of highlighting both the benefits and drawbacks of technological advancements, acknowledging the need for responsible use and governance. This shows an understanding of the complexity of the issue and suggests that the writer has given some thought to the implications of technology on society.

That being said, there are a few areas where the essay could be improved. For example, some of the sentences are quite general and could be more specific and nuanced. Additionally, the transitions between paragraphs could be smoother to create a clearer flow of ideas.

Overall, while the essay has some strengths, it would benefit from more development and refinement to m